# Image Segmentation for Evaluation

* MeanIOU: Image Segmentation에서 많이 쓰이는 evaluation measure
* tf.version 1.12 API: [`tf.metrics.mean_iou`](https://www.tensorflow.org/api_docs/python/tf/metrics/mean_iou)
  * `tf.enable_eager_execution()`이 작동하지 않음
  * 따라서 예전 방식대로 `tf.Session()`을 이용하여 작성하거나 아래와 같이 2.0 version으로 작성하여야 함
* tf.version 2.0 API: [`tf.keras.metrics.MeanIoU`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/metrics/MeanIoU)
* 지금 이 코드는 `version 2` 코드를 이용하여 작성

## Import modules

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os
import time
import shutil
import functools

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['axes.grid'] = False
mpl.rcParams['figure.figsize'] = (12,12)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.image as mpimg
from PIL import Image
from IPython.display import clear_output

import tensorflow as tf

from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models

os.environ["CUDA_VISIBLE_DEVICES"]="1"

## 데이터 수집 및 Visualize

### Download data [트레이닝 시 이미 받음]

이 프로젝트는 [Giana Dataset](https://giana.grand-challenge.org/Dates/)을 이용하여 진행한다.

### Split dataset into train data and test data

In [ ]:
DATASET_PATH='../../datasets/sd_train'

In [ ]:
dataset_dir = os.path.join(DATASET_PATH, 'sd_train')

img_dir = os.path.join(dataset_dir, "train")
label_dir = os.path.join(dataset_dir, "train_labels")

In [ ]:
x_train_filenames = [os.path.join(img_dir, filename) for filename in os.listdir(img_dir)]
x_train_filenames.sort()
y_train_filenames = [os.path.join(label_dir, filename) for filename in os.listdir(label_dir)]
y_train_filenames.sort()

In [ ]:
x_train_filenames, x_test_filenames, y_train_filenames, y_test_filenames = \
                    train_test_split(x_train_filenames, y_train_filenames, test_size=0.2, random_state=219)

In [ ]:
num_train_examples = len(x_train_filenames)
num_test_examples = len(x_test_filenames)

print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples: {}".format(num_test_examples))

## Build our input pipeline with `tf.data`
### Set up test datasets

In [ ]:
# Set hyperparameters
image_size = 256
img_shape = (image_size, image_size, 3)
batch_size = 60

checkpoint_dir = train_dir = 'train/exp1'

In [ ]:
def _process_pathnames(fname, label_path):
  # We map this function onto each pathname pair
  img_str = tf.io.read_file(fname)
  img = tf.image.decode_bmp(img_str, channels=3)

  label_img_str = tf.io.read_file(label_path)
  label_img = tf.image.decode_bmp(label_img_str, channels=1)
  
  resize = [image_size, image_size]
  img = tf.image.resize(img, resize)
  label_img = tf.image.resize(label_img, resize)
  
  scale = 1 / 255.
  img = tf.dtypes.cast(img, tf.float32) * scale
  label_img = tf.dtypes.cast(label_img, tf.float32) * scale
  
  return img, label_img

In [ ]:
def get_baseline_dataset(filenames,
                         labels,
                         threads=5,
                         batch_size=batch_size,
                         shuffle=True):
  num_x = len(filenames)
  # Create a dataset from the filenames and labels
  dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
  # Map our preprocessing function to every element in our dataset, taking
  # advantage of multithreading
  dataset = dataset.map(_process_pathnames, num_parallel_calls=threads)
  
  if shuffle:
    dataset = dataset.shuffle(num_x * 10)
  
  dataset = dataset.batch(batch_size)
  return dataset

In [ ]:
test_dataset = get_baseline_dataset(x_test_filenames,
                                    y_test_filenames,
                                    shuffle=False)

## Build the model

In [ ]:
class Conv(tf.keras.Model):
  def __init__(self, num_filters, kernel_size):
    super(Conv, self).__init__()
    self.conv = layers.Conv2D(num_filters, kernel_size, padding='same')
    self.bn = layers.BatchNormalization()

  def call(self, inputs, training=True):
    x = self.conv(inputs)
    x = self.bn(x, training=training)
    x = tf.nn.relu(x)

    return x

In [ ]:
class ConvBlock(tf.keras.Model):
  def __init__(self, num_filters):
    super(ConvBlock, self).__init__()
    self.conv1 = Conv(num_filters, 3)
    self.conv2 = Conv(num_filters, 3)

  def call(self, inputs, training=True):
    encoder = self.conv1(inputs, training=training)
    encoder = self.conv2(encoder, training=training)

    return encoder


class EncoderBlock(tf.keras.Model):
  def __init__(self, num_filters):
    super(EncoderBlock, self).__init__()
    self.conv_block = ConvBlock(num_filters)
    self.encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))

  def call(self, inputs, training=True):
    encoder = self.conv_block(inputs, training=training)
    encoder_pool = self.encoder_pool(encoder)

    return encoder_pool, encoder


class DecoderBlock(tf.keras.Model):
  def __init__(self, num_filters):
    super(DecoderBlock, self).__init__()
    self.convT = layers.Conv2DTranspose(num_filters, 2, strides=2, padding='same')
    self.bn = layers.BatchNormalization()
    self.conv_block = ConvBlock(num_filters)

  def call(self, input_tensor, concat_tensor, training=True):
    decoder = self.convT(input_tensor)
    decoder = tf.concat([decoder, concat_tensor], axis=-1)
    decoder = self.bn(decoder, training=training)
    decoder = tf.nn.relu(decoder)
    decoder = self.conv_block(decoder, training=training)

    return decoder

In [ ]:
class UNet(tf.keras.Model):
  def __init__(self):
    super(UNet, self).__init__()
    self.encoder_block1 = EncoderBlock(32)
    self.encoder_block2 = EncoderBlock(64)
    self.encoder_block3 = EncoderBlock(128)
    self.encoder_block4 = EncoderBlock(256)

    self.center = ConvBlock(512)

    self.decoder_block4 = DecoderBlock(256)
    self.decoder_block3 = DecoderBlock(128)
    self.decoder_block2 = DecoderBlock(64)
    self.decoder_block1 = DecoderBlock(32)

    self.output_conv = layers.Conv2D(1, 1, activation='sigmoid')

  def call(self, inputs, training=True):
    encoder1_pool, encoder1 = self.encoder_block1(inputs)
    encoder2_pool, encoder2 = self.encoder_block2(encoder1_pool)
    encoder3_pool, encoder3 = self.encoder_block3(encoder2_pool)
    encoder4_pool, encoder4 = self.encoder_block4(encoder3_pool)

    center = self.center(encoder4_pool)

    decoder4 = self.decoder_block4(center, encoder4)
    decoder3 = self.decoder_block3(decoder4, encoder3)
    decoder2 = self.decoder_block2(decoder3, encoder2)
    decoder1 = self.decoder_block1(decoder2, encoder1)

    outputs = self.output_conv(decoder1)

    return outputs

In [ ]:
model = UNet()

## Restore using Checkpoints (Object-based saving)

In [ ]:
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(model=model)

# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

### Display prediction mask image for one test data

In [ ]:
for test_images, test_labels in test_dataset.take(1):
  predictions = model(test_images)
        
  plt.figure(figsize=(10, 20))
  plt.subplot(1, 3, 1)
  plt.imshow(test_images[0,: , :, :])
  plt.title("Input image")

  plt.subplot(1, 3, 2)
  plt.imshow(test_labels[0, :, :, 0])
  plt.title("Actual Mask")

  plt.subplot(1, 3, 3)
  plt.imshow(predictions[0, :, :, 0])
  plt.title("Predicted Mask")
  plt.show()

## Evaluate the test dataset and Plot

In [ ]:
m = tf.keras.metrics.MeanIoU(num_classes=2)

for images, labels in test_dataset:
  predictions = model(images)
  m.update_state( tf.dtypes.cast(tf.math.round(labels), tf.int32),
                  tf.dtypes.cast(tf.math.round(predictions), tf.int32) )

print('Final Mean IOU result: ', m.result().numpy())